In [ ]:
# ------------------------------------------------------------------------------
# Author: SeongIl Kim
# Date: 2025-04-10
# Description: Preprocessing for GT/CT/Stay Time correlation analysis at Busan Port
# ------------------------------------------------------------------------------
import pandas as pd                                      # For data handling (데이터 처리 및 분석)
from pathlib import Path                                 # For setting datapath (데이터 경로 설정)
from sklearn.preprocessing import MinMaxScaler           # For normalization (정규화)

# ------------------------------------------------------------------------------
# Load and clean GT data (총톤수(GT) 데이터 불러오기 및 정리)
# ------------------------------------------------------------------------------
base_path = Path('./useData')
busanPortDf = pd.read_csv(base_path / 'busanport/busan_부산항(전체)_rawData.csv', encoding='utf-8', header=0)
busanPortGT = busanPortDf[['Year', 'GT(Gross Tonnage)']]
busanPortGT = busanPortGT.query('Year < 2025')  # Exclude incomplete 2025 data (2025년 데이터 제외)

# Remove comma and convert GT to integer (쉼표 제거 및 정수형 변환)
busanPortGT['GT(Gross Tonnage)'] = busanPortGT['GT(Gross Tonnage)'].str.replace(',', '', regex=False).astype(int)

# ------------------------------------------------------------------------------
# Load and preprocess CT data (화물처리실적(CT) 데이터 불러오기 및 집계)
# ------------------------------------------------------------------------------
busanPortCT = pd.read_csv(base_path / 'busanCT(화물처리실적).csv', encoding='utf-8', header=0)
busanPortCT_yearly = busanPortCT.groupby('Year').sum().reset_index()[['Year', 'CT(Cargo Throughput)']]

# ------------------------------------------------------------------------------
# Load and process ship schedule data (입출항 스케줄 데이터 처리)
# ------------------------------------------------------------------------------
schedule1 = pd.read_csv(base_path / 'SinhangSchedule_rawData(01_01_2010-31_12_2015).csv', encoding='utf-8')
schedule2 = pd.read_csv(base_path / 'SinhangSchedule_rawData(01_01_2016-31_12_2024).csv', encoding='utf-8')

# Extract year and month from 'Enter Time' (입항 시간에서 연도와 월 추출)
def extract_year_month(df, time_col):
    df['Year'] = df[time_col].str[:4].astype(int)
    df['Month'] = df[time_col].str[5:7].astype(int)
    return df

schedule1 = extract_year_month(schedule1, 'Enter Time')
schedule2 = extract_year_month(schedule2, 'Enter Time')

# Compute stay time in hours (체류시간 계산, 단위: 시간)
schedule1['Stay Time'] = pd.to_datetime(schedule1['Out Time']) - pd.to_datetime(schedule1['Enter Time'])
schedule2['Stay Time'] = pd.to_datetime(schedule2['Out Time']) - pd.to_datetime(schedule2['Enter Time'])
schedule1['Stay Time'] = round(schedule1['Stay Time'].dt.total_seconds() / 3600, 1)
schedule2['Stay Time'] = round(schedule2['Stay Time'].dt.total_seconds() / 3600, 1)

# Filter data from 2013 onward (2013년 이후 데이터만 사용)
schedule1 = schedule1.query('Year >= 2013')

# Select required columns for analysis (필요한 열만 추출)
schedule1 = schedule1[['Year', 'Month', 'Stay Time']]
schedule2 = schedule2[['Year', 'Month', 'Stay Time']]

# Merge both schedule datasets (스케줄 데이터 병합)
schedule_all = pd.concat([schedule1, schedule2], axis=0, ignore_index=True)

# Aggregate monthly total stay time (월별 체류시간 합계 계산)
busanPortStayTime = schedule_all.groupby(['Year', 'Month']).agg({'Stay Time': 'sum'}).reset_index()

# ------------------------------------------------------------------------------
# Merge yearly GT and CT data (연도별 GT와 CT 데이터 병합)
# ------------------------------------------------------------------------------
busanPortCorrDf = pd.merge(busanPortGT, busanPortCT_yearly, on='Year')

# Save raw yearly correlation data (원본 연도별 상관분석 데이터 저장)
busanPortCorrDf.to_csv(base_path / 'busanPortCorrRaw.csv', encoding='utf-8-sig')

# ------------------------------------------------------------------------------
# Merge monthly CT and stay time data (월별 CT와 체류시간 데이터 병합)
# ------------------------------------------------------------------------------
busanPortCorrDf_monthly = pd.merge(busanPortCT, busanPortStayTime, on=['Year', 'Month'])

# Save raw monthly correlation data (원본 월별 상관분석 데이터 저장)
busanPortCorrDf_monthly.to_csv(base_path / 'busanPortCorrMonthlyRaw.csv', encoding='utf-8-sig')

# ------------------------------------------------------------------------------
# Normalize data using MinMaxScaler (MinMax 정규화 적용)
# ------------------------------------------------------------------------------
scaler_gt = MinMaxScaler()
scaler_ct1 = MinMaxScaler()
scaler_ct2 = MinMaxScaler()
scaler_stay_time = MinMaxScaler()

busanPortCorrDf['GT(Gross Tonnage)'] = scaler_gt.fit_transform(busanPortCorrDf[['GT(Gross Tonnage)']])
busanPortCorrDf['CT(Cargo Throughput)'] = scaler_ct1.fit_transform(busanPortCorrDf[['CT(Cargo Throughput)']])
busanPortCorrDf_monthly['CT(Cargo Throughput)'] = scaler_ct2.fit_transform(busanPortCorrDf_monthly[['CT(Cargo Throughput)']])
busanPortCorrDf_monthly['Stay Time'] = scaler_stay_time.fit_transform(busanPortCorrDf_monthly[['Stay Time']])

# ------------------------------------------------------------------------------
# Save normalized datasets (정규화된 데이터 저장)
# ------------------------------------------------------------------------------
busanPortCorrDf.to_csv(base_path / 'busanPortCorrScaled.csv', encoding='utf-8-sig')
busanPortCorrDf_monthly.to_csv(base_path / 'busanPortCorrMonthlyScaled.csv', encoding='utf-8-sig')